In [ ]:
from paddleocr import PaddleOCR
import numpy as np
from sklearn.cluster import KMeans

# OCR 모델 초기화 -> False로 한 이유는 속도 향상으로
ocr = PaddleOCR(
    use_doc_orientation_classify=False,  # 문서 기울기 파악해서 보정할건지
    use_doc_unwarping=False,             # 문서 구겨지거나 왜곡된거 펼건지
    use_textline_orientation=False,      # 글자 한 줄 한 줄 기울기 파악해서 보정할건지 
    lang="korean"
)

def extract_text(image_path, y_threshold=15, diff_threshold=600, balance_ratio=0.4):  # y_threshold=15, diff_threshold=600, balance_ratio=0.4 
    results = ocr.predict(input=image_path)
    
    if not results or not results[0]:
        return []

    result = results[0]
    boxes = result.get("rec_boxes", [])
    texts = result.get("rec_texts", [])
    
    if isinstance(boxes, np.ndarray):
        boxes = boxes.tolist()
        
    if not boxes:
        return []
    
    center_points = []
    for box in boxes:
        pts = np.array(box).reshape(-1, 2)
        cx = np.mean(pts[:, 0])
        cy = np.mean(pts[:, 1])
        center_points.append((cx, cy))

    X = np.array([[cx] for cx, _ in center_points])

    # K-Means를 통해 칼럼을 감지하는 로직
    is_column_layout = False
    labels = [0] * len(center_points)
    
    if len(center_points) > 4:
        kmeans = KMeans(n_clusters=2, random_state=0, n_init=10).fit(X)
        labels = kmeans.labels_
        centers = kmeans.cluster_centers_.flatten()
        diff = abs(centers[0] - centers[1])

        counts = [sum(labels == i) for i in range(2)]
        left_ratio = min(counts) / max(counts) if max(counts) > 0 else 0

  
        if diff > diff_threshold and left_ratio >= balance_ratio:
            is_column_layout = True

    def sort_and_group(items, y_threshold=15):
        items.sort(key=lambda x: (x[0], x[1]))
        grouped, current_line, last_y = [], [], None
        for cy, cx, text in items:
            if last_y is None or abs(cy - last_y) <= y_threshold:
                current_line.append((cx, text))
            else:
                current_line.sort(key=lambda x: x[0])
                grouped.append(" ".join([t for _, t in current_line]))
                current_line = [(cx, text)]
            last_y = cy
        if current_line:
            current_line.sort(key=lambda x: x[0])
            grouped.append(" ".join([t for _, t in current_line]))
        return grouped

    if is_column_layout:
        left_label = np.argmin(centers)
        left_items, right_items = [], []
        for (cx, cy), text, label in zip(center_points, texts, labels):
            if label == left_label:
                left_items.append((cy, cx, text))
            else:
                right_items.append((cy, cx, text))
        left_lines = sort_and_group(left_items, y_threshold)
        right_lines = sort_and_group(right_items, y_threshold)
        ordered_texts = left_lines + right_lines
    else:
        items = [(cy, cx, text) for (cx, cy), text in zip(center_points, texts)]
        ordered_texts = sort_and_group(items, y_threshold)

    return ordered_texts

# 테스트
image1 = "C:/Potenup/Drug-Detection-Chatbot/data/medicine_00451.jpeg"
image2 = "C:/Potenup/Drug-Detection-Chatbot/data/medicine_00685.jpg"

print("==== 첫번째 이미지 ====")
for line in extract_text(image1):
    print(line)

print("\n==== 두번째 이미지 ====")
for line in extract_text(image2):
    print(line)

Creating model: ('PP-OCRv5_server_det', None)


Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\user\.paddlex\official_models\PP-OCRv5_server_det`.
Creating model: ('korean_PP-OCRv5_mobile_rec', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\user\.paddlex\official_models\korean_PP-OCRv5_mobile_rec`.


==== 첫번째 이미지 ====
[원료약품 및 그 분량]19 중
·유효성분:
디플루코르톨론발레레이트
(BP) 3mg
첨가제(보존제):
파라옥시벤조산메틸(KP)
1.8mg
파라옥시벤조산프로필(KP)
.0.2mg
기타첨가제:
경질유동파라핀,라우릴황산
나트륨,모노스테아르산소르
비탄,세탄올,스테아릴알코
올,정제수,카보머940,트
롤아민,프로필렌글리콜
[성상]
흰색~미담황색의 균질한 로션제
[효능·효과]
첨부문서참조
[용법·용량]
1일 2~3회 앞게  바른다.
증상이 호전되면 1일 1회로
충분하다.

==== 두번째 이미지 ====
케이 캡정
[원료약품 및그분량]이약1정(206mg) 중
유효성분:테고프라잔(별규)
기타 첨가제:D-만니톨
히드록시프로필셀룰로오스.
오파드라이1분홍색(85F240134)
장방혀
[성상]연한 분홍색의
O00
[효능 효과]1.미란성
2.비미란성
3.위궤양의 치료
4.소화성 궤양
박터파일로리
[용법·용량]이 약은 성인에게
1.미란성위식도역류질환의 치료:1일1회
식도염이 치료되지 않거나
21 .비미란성
위궤양의 치료:1일1회1회 50mg을 8주간 경구투여한다.
위한 항생제
받아야 한다.
1일2회 7일간 경구투여한다
이약은 식사와 관계없이 투여할 수 있다.
[사용상의 주의사항] 첨부문서 참조
[저장방법]기밀용기,실온(1~30'C)보관
R
K-CAB
Tegoprazan 50mg
50.0mg
미결정셀룰로오스, 크로스카르멜로오스나트륨.
콜로이드성이산화규소. 스테아르산마그네슘.
필름코팅정
위식도역류질환의 치료
위식도역류질환의치료
및/또는 만성 위축성 위염 환자에서의 헬리코
제균을 위한 항생제 벼요요번
이능프급
다음과 같이 투여한다
1회 50mg을 4주간 경구투여한다.
증상이 계속되는 환자의 경우4주 더투여한다.
위식도역류질환의 치료:1일 1회,1회 50mg을 4주간 경구투여한다.
4.소화성 궤양 및/또는 만성 위축성 위염 환자에서의 헬리코박터파일로리 제균을
병용요법:헬리코박터파일로리 감염환자들은 제균요법으로 치료
이약50mg과 아목시실린1g.클래리트로마이신5